---
jupyter:
  jupytext:
    formats: md,ipynb
    text_representation:
      extension: .md
      format_name: markdown
      format_version: '1.3'
      jupytext_version: 1.16.0
  kernelspec:
    display_name: Python 3 (ipykernel)
    language: python
    name: python3
---

<!-- #region id="549d9fb9" -->
# Table des matières
1. [La culture de l'expérimentation](#la-culture-de-lexpérimentation)
1. [Le b.a.-ba des méthodes de projections](#le-b.a.-ba-des-méthodes-de-projections)
1. [Exemples de méthodes de réduction de la dimensionnalité](#exemples-de-méthodes-de-réduction-de-la-dimensionnalité)
  1. [Lecture et affichage des données](#lecture-et-affichage-des-données)
  1. [Projection aléatoire](#projection-aléatoire)
  1. [Analyse en composantes principales](#analyse-en-composantes-principales)
  1. [Analyse par discriminants linéaires](#analyse-par-discriminants-linéaires)
  1. [Isomap](#isomap)
  1. [Encodage localement linéaire](#encodage-localement-linéaire)
  1. [Échelonnage multidimensionnel métrique](#échelonnage-multidimensionnel-métrique)
  1. [Encodage spectral](#encodage-spectral)
  1. [*T-Distributed Stochastic Neighbor Embedding*](#it-distributed-stochastic-neighbor-embeddingi)
1. [Applications](#applications)
  1. [Déroulement d'un rouleau suisse](#déroulement-dun-rouleau-suisse)
  1. [Aplatissement d'une sphère manquant un quartier](#aplatissement-dune-sphère-manquant-un-quartier)
<!-- #endregion -->



In [ ]:
from sklearn.utils import check_random_state
from sklearn import (
    manifold,
    datasets,
    decomposition,
    ensemble,
    discriminant_analysis,
    random_projection,
)
from sklearn.decomposition import FastICA
from collections import OrderedDict
from functools import partial
from time import time
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.ticker import NullFormatter

import numpy as np
import matplotlib.pyplot as plt
import warnings

Axes3D

seed = 42
np.random.seed(seed)



<!-- #region id="d8ea47b9" -->
La réduction de la dimensionnalité consiste à prendre des données dans un espace de grande dimension, et à les
transformer en données dans un espace de plus petite dimension. Le défi consiste à conserver le maximum d’information
partagée entre les données lors de la transformation. Ainsi, les données en sortie, plus petites, pourront
être traitées plus rapidement et plus efficacement avec nos algorithmes d’analyse et de traitement.
Les applications sont multiples en classification, régression et regroupement des données (*Data Clustering*).

Une des raisons pour lesquelles la réduction de la dimensionnalité est si importante est liée au fait
que les algorithmes performent mal avec des données en haute dimension. Le problème est connu sous le nom du
[fléau de la dimension](https://fr.wikipedia.org/wiki/Fléau_de_la_dimension#:~:text=Le_fléau_de_la_dimension_ou_malédiction_de,pas_lieu_dans_des_espaces_dimension_moindre.)
 (*Curse of Dimensionnality*).

On ne peut pas simplement enlever plusieurs des dimensions d'un ensemble de données afin de réduire la dimension
finale de celles-ci. La figure suivante montre ce qu'il arrive lorsqu'on projette une surface 3-D sur une surface 2-D en éliminant une composante à la fois. Chaque objet en bois se transforme en différentes lettres selon la direction de projection! Il y a beaucoup d'information perdue à chaque fois qu'on élimine une des dimensions.
<!-- #endregion -->

<!-- #region id="127a1ffc" -->
<p>&nbsp;</p>
<div align="center">
    <img src= "../images/godel-escher-bach.jpeg"  width="250" />
    <div>
    <font size="0.5">Image Source: https://emergentbydesign.com/2012/04/16/what-are-the-most-life-changing-books-youve-read-twitter-poll//</font>
    </div>
</div>
<p>&nbsp;</p>
<!-- #endregion -->

<!-- #region id="92adbb6b" -->
Plusieurs méthodes de réduction de la dimensionnalité procèdent de façon plus astucieuse. On transforme d'abord
l'ensemble de données initial $X=[x_{1}, \cdots, x_{N}]$ en un nouvel ensemble $Y=[y_{1}, \cdots, y_{N}]$ où
la majorité de l'information est contenue dans les premières composantes; $y_{1}, y_{2}$ par exemple. Idéalement,
les points voisins dans l'espace de départ devraient rester voisins dans l'espace d'arrivée. Puis, on ne garde que les $M$ premières composantes du vecteur transformé, soit $Y=[y_{1}, \cdots, y_{M}]$ avec $M<<N$. La valeur de $M$ est déterminée  selon la méthode utilisée.

Le panneau gauche de la figure suivante montre un exemple 3-D où les données occupent les trois dimensions, mais sont
regroupées sur une surface de plus petite dimension (*Manifold*), dans ce cas-ci un plan. Le panneau de droite
montre le résultat de la réduction de la dimensionnalité. Dans cet exemple, on a utilisé
l'[analyse en composantes principales (ACP)](https://fr.wikipedia.org/wiki/Analyse_en_composantes_principales) 
pour transformer les données. C'est une des méthodes présentées dans ce module. Notez comme
le bruit dans les données a été réduit en passant de 3-D en 2-D; cela faciliterait
par exemple la classification des quatre nuages de points.

Les méthodes précédentes sont linéaires; elles projettent les données d'un espace N-D vers un autre espace N-D, plus pratique, puis en extraient un sous-ensemble de $M$ variables. Les méthodes non linéaires sont encore plus astucieuses. Elles transforment directement un espace N-D en un autre M-D avec $M<<N$; ce n'est pas une projection, mais plutôt un plongement (*Data Embedding*). Les deux types de méthodes sont présentés dans le module.
<!-- #endregion -->

<!-- #region colab_type="text" id="103f39a5" -->
<p>&nbsp;</p>
<div align="center">
    <img src="../images/pca-illustration.png"  width="700" />
    <div>
    <font size="0.5">Image Source: http://phdthesis-bioinformatics-maxplanckinstitute-molecularplantphys.matthias-scholz.de/</font>
    </div>
</div>
<p>&nbsp;</p>

<!-- #endregion -->

<!-- #region id="c0b0befb" -->
# <a id=la-culture-de-lexpérimentation>La culture de l'expérimentation</a>
<!-- #endregion -->

<!-- #region id="828072db" -->
<p>&nbsp;</p>
<div align="center">
    <img src= "../images/baby-lego.jpeg"  width="250" />                                                       <div>
    <font size="0.5">Image Source: https://p1.pxfuel.com/preview/632/789/181/baby-lego-boy-constructor.jpg/</font>
    </div>
</div>

<!-- #endregion -->

<!-- #region id="e1ff1032" -->
On présente dans ce module plusieurs méthodes de réduction de la dimensionnalité sans explications
(PCA, MDS, T-SNE, etc.). On les introduit uniquement comme méthodes possibles à essayer. Elles sont
populaires et très utilisées en analyse des données. Expliquer ici chacune d’entre elles serait
trop long. Si vous voulez en savoir plus, nous vous invitons à lire à leur sujet en consultant par
exemple la documentation de Scikit-learn.

En pratique, on essaie souvent plusieurs méthodes pour solutionner un problème. Celles qui fonctionnent
bien sont ensuite analysées plus à fond. Pas besoin, dans un premier temps, de connaitre la théorie
derrière chaque méthode possiblement intéressante. C’est un peu l’idée de Scikit-learn qui montre plein
de démos utiles. On essaie celles qui semblent prometteuses et on approfondit ensuite celles qu’on
a finalement décidé d’utiliser.

<!-- #endregion -->

<!-- #region colab_type="text" id="2afbd2c6" -->
# <a id=le-b.a.-ba-des-méthodes-de-projections>Le b.a.-ba des méthodes de projections</a>
<!-- #endregion -->

<!-- #region id="944b3168" -->
Dans la section suivante, nous allons utiliser la réduction de la dimensionnalité afin de regrouper
ensemble des images similaires de dimension 8x8. C'est une application simple qui permet de bien comprendre l'idée
derrière la méthode.

Supposons que nous avons un ensemble d'images telles que chacune correspond à un point dans un espace de
dimension $8\times8=64$. Il est évidemment impossible de visualiser la distribution des images sous cette forme.
On peut toutefois réduire la dimensionnalité de l'espace afin de le projeter dans un sous-espace de dimension 2. C'est
nettement plus pratique!

Parler de projeter une image dans un espace 2-D (deux dimensions) est évidemment un abus de langage.
Cela veut simplement dire que pour chaque image, prenons celle d'un 4 par exemple, on transforme ses coordonnées de
64-D à 2-D, puis on affiche le symbole 4 à la position 2-D dans le plan. On ne s'intéresse
pas à une déformation quelconque de l'image du 4, mais uniquement à sa position finale dans l'espace d'arrivée en 2-D.

Il existe de nombreuses méthodes de projection produisant des résultats souvent
très différents. Nous allons en décrire plusieurs. Idéalement, les chiffres similaires
devraient tous être regroupés ensemble, sans superposition avec d'autres chiffres, dans le plan 2-D.

Plusieurs des méthodes sont linéaires et d'autres sont non linéaires. Qu'est-ce que ça veut dire? Commençons
par les méthodes linéaires. Transformons une image, par exemple le chiffre 4 précédent, en
un vecteur $X$ en 64-D. Une projection linéaire transformera le vecteur $X$ en un nouveau vecteur $Y$ en 64-D
selon

$$\bf{y} =  \bf{M} \bf{x}$$
avec

$$\bf{M} = \begin{pmatrix} m_{1,1} & m_{1,2} & \cdots & x_{1,64} \\
\vdots & &  &   \vdots \\
m_{64,1} & x_{64,2} & \cdots & x_{64,64} &  \end{pmatrix},
\bf{x} =\begin{pmatrix} x_{1} \\ x_{2} \\ \vdots \\ x_{64}  \end{pmatrix},
     \bf{y} =\begin{pmatrix} y_{1} \\ y_{2} \\ \vdots \\ y_{64}  \end{pmatrix}.$$
L'étape suivante consiste à ne garder que les deux premières valeurs du vecteur transformé $Y$. L'affichage des
points $(y_{1},y_{2})$ dans le plan produira plusieurs des figures qui suivent. Pour chaque méthode de réduction de
dimensionnalité, le secret de la sauce est dans les coefficients de la matrice de transformation $M$.

Les méthodes non linéaires ne sont pas basées sur des combinaisons
linéaires des composantes $x_{i}$. Elles sont plus complexes et généralement itératives. C'est-à-dire, qu'il n'existe pas de
relation simple entre les composantes $x_{i}$ et $y_{i}$. Difficile d'être plus simple sans aller dans les détails.

Les méthodes linéaires ont été les premières utilisées, car on peut les dériver analytiquement. Les méthodes
non linéaires, généralement plus performantes, ont émergé des travaux
sur les réseaux de neurones et ont été mises en pratique grâce à la puissance de calcul des ordinateurs.

Plusieurs méthodes sont présentées dans ce qui suit. Les méthodes linéaires sont présentées en premier.
Les méthodes non linéaires suivent ensuite. Elles donnent généralement de
meilleurs résultats que les premières, mais sont plus gourmandes en mémoire et temps de calcul.
<!-- #endregion -->

<!-- #region id="8e8dc267" -->
# <a id=exemples-de-méthodes-de-réduction-de-la-dimensionnalité>Exemples de méthodes de réduction de la dimensionnalité</a>
<!-- #endregion -->

<!-- #region id="5b85ee64" -->
Cette partie est inspirée des
[exemples de code](https://scikit-learn.org/stable/auto_examples/manifold/plot_lle_digits.html#sphx-glr-auto-examples-manifold-plot-lle-digits-py)
de la librairie Scikit-learn.
<!-- #endregion -->

<!-- #region id="38feb164" -->
### <a id=lecture-et-affichage-des-données>Lecture et affichage des données</a>
<!-- #endregion -->

<!-- #region id="ca2d21f6" -->
Nous allons utiliser un sous-ensemble du jeu de données
[**MNIST**](https://en.wikipedia.org/wiki/MNIST_database)
qui comprend des images de chiffres 0 à 9 de taille $8 \text{ par } 8$. Les images du jeu de données original sont de taille $28 \text{ par } 28$. Nous allons n'utiliser que les images des chiffres 0 à 5.
<!-- #endregion -->



In [ ]:
digits = datasets.load_digits(n_class=6)
X = digits.data
y = digits.target
n_samples, n_features = X.shape
n_neighbors = 30
n_components = 2



<!-- #region colab_type="text" id="2afbd2c6" -->
Exemples d'images de chiffres 0 à 5.
<!-- #endregion -->



In [ ]:
n_img_per_row = 20
img = np.zeros((10 * n_img_per_row, 10 * n_img_per_row))
for i in range(n_img_per_row):
    ix = 10 * i + 1
    for j in range(n_img_per_row):
        iy = 10 * j + 1
        img[ix : ix + 8, iy : iy + 8] = X[i * n_img_per_row + j].reshape((8, 8))

plt.imshow(img, cmap=plt.cm.binary)
plt.xticks([])
plt.yticks([])
plt.title('Une sélection des 6 premiers chiffres du jeu de données MNIST');



<!-- #region id="38feb164" -->
Le code ci-dessous sert à afficher les graphiques pour les exemples de projection qui suivent.
<!-- #endregion -->



In [ ]:
def plot_embedding(X, title=None):
    x_min, x_max = np.min(X, 0), np.max(X, 0)
    X = (X - x_min) / (x_max - x_min)

    plt.figure()
    ax = plt.subplot(111)
    for i in range(X.shape[0]):
        plt.text(
            X[i, 0],
            X[i, 1],
            str(y[i]),
            color=plt.cm.Set1((y[i] + 1) / 10.0),
            fontdict={'weight': 'bold', 'size': 9},
        )
    plt.xticks([]), plt.yticks([])
    if title is not None:
        plt.title(title)



<!-- #region colab_type="text" id="0cfcd114" -->
## <a id=projection-aléatoire>Projection aléatoire</a>
<!-- #endregion -->

<!-- #region id="c743755b" -->
Cette méthode effectue une projection des chiffres au moyen d'une matrice aléatoire clairsemée (*Sparse Random Matrix*).
<!-- #endregion -->



In [ ]:
%%time
rp = random_projection.SparseRandomProjection(
    n_components=n_components, random_state=seed
)
X_projected = rp.fit_transform(X)
plot_embedding(X_projected, "Projection aléatoire des chiffres")



<!-- #region id="5a451fb7" -->
Les résultats ne sont pas particulièrement convaincants puisqu'on observe un grand recouvrement des 
classes de chiffres; la séparation des images 0 à 5 est inefficace.
<!-- #endregion -->

<!-- #region colab_type="text" id="6a7be59a" -->
## <a id=analyse-en-composantes-principales>Analyse en composantes principales</a>
<!-- #endregion -->

<!-- #region id="4dccd501" -->
L'analyse en composantes principales (*Principal Components Analysis* ou *PCA*) est une méthode linéaire classique qui consiste à transformer des variables $x_i$ corrélées entre elles (c.-à-d. liées) en nouvelles variables $y_i$ décorrélées les unes des autres. Les nouvelles variables sont nommées « composantes principales ». Elles sont ordonnées en ordre décroissant d'importance (mesurée par leur variance) selon

$$\sigma^2(y_1)>\sigma^2(y_2)>\cdots \sigma^2(y_{N})$$

Lorsqu'on veut compresser un ensemble de $N$ variables aléatoires, les $M$ premiers axes de l'analyse en composantes principales sont un meilleur choix, du point de vue de la variance.

La figure suivante montre les deux premiers axes principaux.

<!-- #endregion -->



In [ ]:
%%time
X_pca = decomposition.PCA(n_components=n_components).fit_transform(X)
plot_embedding(X_pca, "Projection des chiffres en composantes principales")



<!-- #region id="9ace3a71" -->
La séparation des classes de chiffres est meilleure que pour la projection aléatoire.
<!-- #endregion -->

<!-- #region colab_type="text" id="1883db4e" -->
## <a id=analyse-par-discriminants-linéaires>Analyse par discriminants linéaires</a>
<!-- #endregion -->

<!-- #region id="8930322e" -->
L'analyse par discriminants linéaires (*Linear Discriminant Analysis* ou *LDA*) est une méthode linéaire qui projette les chiffres au moyen des deux principaux discriminants linéaires. La transformation utilise à la fois les positions des données et leurs étiquettes. Elle surpasse la PCA lorsqu'il y a beaucoup de données, autrement la PCA est à privilégier.
<!-- #endregion -->



In [ ]:
%%time
X2 = X.copy()
X2.flat[:: X.shape[1] + 1] += 0.01  # X doit être inversable ...
X_lda = discriminant_analysis.LinearDiscriminantAnalysis(
    n_components=n_components
).fit_transform(X2, y)
plot_embedding(X_lda, "Projection des chiffres par discrimination linéaire")



<!-- #region id="90d78728" -->
Les regroupements observés sont les meilleurs jusqu'à maintenant.
<!-- #endregion -->

<!-- #region colab_type="text" id="52fe5db8" -->
## <a id=isomap>Isomap</a>
<!-- #endregion -->

<!-- #region id="8c5324fc" -->
Cette méthode et les suivantes sont toutes non linéaires.
<!-- #endregion -->



In [ ]:
%%time
X_iso = manifold.Isomap(
    n_neighbors=n_neighbors, n_components=n_components
).fit_transform(X)
plot_embedding(X_iso, "Projection des chiffres par Isomap")



<!-- #region colab_type="text" id="df6a54d6" -->
## <a id=encodage-localement-linéaire>Encodage localement linéaire</a>
<!-- #endregion -->

<!-- #region id="d62c240a" -->
Si on aime jouer avec les mots, on peut dire qu'une méthode localement linéaire est globalement non linéaire.
<!-- #endregion -->



In [ ]:
%%time
clf = manifold.LocallyLinearEmbedding(
    n_neighbors=n_neighbors, n_components=n_components, method='standard'
)
X_lle = clf.fit_transform(X)
plot_embedding(X_lle, "Encodage localement linéaire des chiffres")



<!-- #region id="554511ca" -->
Pas très convaincant dans ce cas-ci...
<!-- #endregion -->

<!-- #region colab_type="text" id="d44e01f0" -->
## <a id=échelonnage-multidimensionnel-métrique>Échelonnage multidimensionnel métrique</a>
<!-- #endregion -->



In [ ]:
%%time
clf = manifold.MDS(n_components=n_components, n_init=1, max_iter=100)
X_mds = clf.fit_transform(X)
plot_embedding(X_mds, "Encodage MDS des chiffres")



<!-- #region id="ea81db6a" -->
Les classes sont assez bien séparées, mais ont une grande dispersion interne.
<!-- #endregion -->

<!-- #region colab_type="text" id="6ecd38a5" -->
## <a id=encodage-spectral>Encodage spectral</a>
<!-- #endregion -->



In [ ]:
%%time
embedder = manifold.SpectralEmbedding(
    n_components=n_components, random_state=0, eigen_solver="arpack"
)
X_se = embedder.fit_transform(X)

plot_embedding(X_se, "Encodage spectral des chiffres")



<!-- #region id="d99b06d5" -->
Les résultats suivants ne sont pas facilement interprétables, bien que les chiffres soient relativement bien séparés.
<!-- #endregion -->

<!-- #region colab_type="text" id="f492f4cd" -->
# <a id=it-distributed-stochastic-neighbor-embeddingi>*T-Distributed Stochastic Neighbor Embedding*</a>
Cette méthode est souvent connu sous le nom t-SNE.
<!-- #endregion -->



In [ ]:
%%time
tsne = manifold.TSNE(n_components=n_components, init='pca', random_state=0)

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    X_tsne = tsne.fit_transform(X)

plot_embedding(X_tsne, "Encodage t-SNE des chiffres")



<!-- #region id="0dd64ba8" -->
C'est la meilleure avec ce jeu de données. Non seulement les classes sont bien séparées, mais elles ont une faible
dispersion interne; les nuages de points sont compacts. C'est une des méthodes les plus populaires
en ce moment. La séparation est rarement parfaite, mais elle est souvent meilleure que
celles obtenues avec les autres méthodes.

Regardez le site suivant pour une présentation de la méthode
[t-SNE](https://github.com/oreillymedia/t-SNE-tutorial) avec animation.

> À noter que la méthode t-SNE peut afficher des liens différents selon les hyperparamètres utilisés. Cela veut dire que les résultats observés peuvent changer; les regroupements peuvent être moins bien séparés et même se superposer partiellement.
Regardez-le [site suivant](https://distill.pub/2016/misread-tsne/) pour en savoir plus sur les forces et les faiblesses de la méthode.

<!-- #endregion -->

<!-- #region colab={} colab_type="code" id="4ecc1c90" -->
# <a id=applications>Applications</a>
<!-- #endregion -->

<!-- #region id="df6283f0" -->
Une application surprenante de la réduction de la dimensionnalité est l'aplatissement ou le déroulement
d'une surface 3-D sur une surface 2-D, par exemple sur une table. Nous allons en voir deux exemples.
<!-- #endregion -->

<!-- #region id="1ed38c62" -->
Le code ci-dessous sert à afficher les graphiques.
<!-- #endregion -->



In [ ]:
def DerouleSurface3D(X, colors, n_neighbors=10):
    fig = plt.figure(figsize=(15, 8))
    fig.suptitle(
        "Déroulement ou aplatissement de surface 3D avec \
                 %i points, %i voisins"
        % (X.shape[0], n_neighbors),
        fontsize=14,
    )

    # Affiche la surface 3 D de départ
    ax = fig.add_subplot(231, projection='3d')
    ax.scatter(X[:, 0], X[:, 1], X[:, 2], c=colors, cmap=plt.cm.Spectral)
    ax.view_init(4, -72)

    # Méthodes de réduction de dimensionnalité utilisées pour effectuer le
    # déroulement de la surface sur un plan.
    LLE = partial(
        manifold.LocallyLinearEmbedding,
        n_components=2,
        n_neighbors=n_neighbors,
        eigen_solver='auto',
    )

    methods = OrderedDict()
    methods['Encodage localement linéaire'] = LLE(method='standard')
    methods['Isomap'] = manifold.Isomap(n_components=2, n_neighbors=n_neighbors)
    methods['MDS'] = manifold.MDS(n_components=2, max_iter=100, n_init=1)
    methods['Encodage spectral'] = manifold.SpectralEmbedding(
        n_components=2, n_neighbors=n_neighbors
    )
    methods['t-SNE'] = manifold.TSNE(n_components=2, init='pca', random_state=0)

    # Affiche le résultat du déroulement pour chaque méthode
    for i, (label, method) in enumerate(methods.items()):
        t0 = time()
        with warnings.catch_warnings():
            warnings.simplefilter("ignore")
            Y = method.fit_transform(X)
        t1 = time()
        print("%s: %.2g sec" % (label, t1 - t0))
        ax = fig.add_subplot(2, 3, 2 + i)
        ax.scatter(Y[:, 0], Y[:, 1], c=colors, cmap=plt.cm.Spectral)
        ax.set_title("%s" % (label))
        ax.xaxis.set_major_formatter(NullFormatter())
        ax.yaxis.set_major_formatter(NullFormatter())
        ax.axis('tight')

    plt.show()



<!-- #region id="62180a35" -->
## <a id=déroulement-dun-rouleau-suisse>Déroulement d'un rouleau suisse</a>
<!-- #endregion -->

<!-- #region id="fb278b60" -->
<p>&nbsp;</p>
<div align="center">
    <img src= "../images/vesuvius-scrolls.jpeg"  width="500" />
    <div>
    <font size="0.5">Image Source: https://allthatsinteresting.com/virtually-unravel-vesuvius-scrolls/</font>
    </div>
</div>

<!-- #endregion -->

<!-- #region id="f36f3b0d" -->
Voici une application de la réduction de dimensionnalité en archéologie. On utilise cette approche parmi
d'autres, pour dérouler des rouleaux de papyrus carbonisés! Un grand nombre d'entre eux ont
été trouvés dans une bibliothèque de la ville d'Herculanum détruite lors de l'éruption du Vésuve, en Italie,
en 79 A.D. Ils ont été imagés dans un scanneur CT (à rayons X) et on peut voir l'encre disposée en rouleau suisse
dans sa matrice carbonisée! On peut isoler les positions 3-D des zones d'encre puis utiliser plusieurs des
méthodes précédentes afin de projeter les zones d'encre en 2-D; on peut alors lire un papyrus carbonisé
il y a près de 2 000 ans! Comme dans bien des choses, le principe est simple, mais les détails sont complexes. Regardez la
[vidéo](https://youtu.be/PpNq2cFotyY) suivante pour vous donner une idée des défis à relever.

La figure suivante montre les résultats du déroulement du rouleau suisse obtenus en utilisant cinq des
méthodes présentées dans la première section. Le premier panneau montre la surface de départ et les autres
montrent les résultats du déroulement. Le code de couleur identifie les points voisins avant et après chaque
transformation. Un bon déroulement de la surface devrait conserver l'agencement local des couleurs.

Cet exemple s'inspire d'un exemple de code de la librairie
[Scikit-learn](https://scikit-learn.org/stable/auto_examples/manifold/plot_compare_methods.html#sphx-glr-auto-examples-manifold-plot-compare-methods-py).

<!-- #endregion -->



In [ ]:
n_points = 1000
n_neighbors = 10

# Génération du rouleau suisse
X, couleurs = datasets.make_swiss_roll(n_points)

DerouleSurface3D(X, couleurs, n_neighbors)



<!-- #region id="3073a9e2" -->
La méthode Isomap produit les meilleurs résultats. Notez que la méthode t-SNE, qui marchait si bien
dans la section précédente, déchire la bande en plusieurs sections.

La méthode Isomap devrait permettre de lire quelques sections d'un papyrus déroulé.
<!-- #endregion -->

<!-- #region id="1133a714" -->
## <a id=aplatissement-dune-sphère-manquant-un-quartier>Aplatissement d'une sphère manquant un quartier</a>
<!-- #endregion -->

<!-- #region id="4af03e52" -->
À première vue, ça semble impossible à faire. Toutefois, il ne faut pas oublier que les méthodes de réduction de
dimensionnalité peuvent tordre les objets, comme on l'a vu dans l'exemple précédent. L'exemple qui suit
s'inspire d'un exemple de code de la librairie
[Scikit-learn](https://scikit-learn.org/stable/auto_examples/manifold/plot_manifold_sphere.html#sphx-glr-auto-examples-manifold-plot-manifold-sphere-py/).
<!-- #endregion -->



In [ ]:
n_points = 2000
n_neighbors = 30

# Génération d'une sphère manquant un quartier
random_state = check_random_state(0)
phi = random_state.rand(n_points) * (1.5 * np.pi)
theta = random_state.rand(n_points) * np.pi

x, y, z = np.sin(theta) * np.cos(phi), np.sin(theta) * np.sin(phi), np.cos(theta)

X = np.array([x, y, z]).T
couleurs = phi

DerouleSurface3D(X, couleurs, n_neighbors)



<!-- #region id="28b5f947" -->
Toutes les méthodes, sauf t-SNE qui la déchire, réussissent à déformer localement la sphère 3-D afin
de la déplier sur un plan! C'est un exercice de topologie facile à réaliser.

Pensez-y un instant. Ces méthodes utilisent uniquement les notions de voisinage pour déformer des
surfaces complexes. Aucun modèle analytique de sphère ou de rouleau suisse n'est utilisé, sauf
pour générer les données initiales.

Libre à vous d'explorer de nouvelles formes (non fermées) à déformer!
<!-- #endregion -->
